In [ ]:
from typing_extensions import TypedDict, List, Optional, Union, Dict, Annotated, Literal
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, trim_messages
from langchain_core.documents import Document
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START, MessagesState
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from team_tools import tavily_search_tool, arxiv_search_tool, web_scraper_tool, repl_tool
from qdrant_cloud_ops import initialize_selfquery_retriever, qdrant_vector_store
from llm_chains import decomposition_chain, requires_decomposition, rephrase_chain, get_plan_chain, assign_chat_topic
from dotenv import load_dotenv
from pprint import pprint
import re
import functools
import operator
from langgraph.checkpoint.memory import MemorySaver

from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

from token_counter import tiktoken_counter




load_dotenv()

in_memory_store = InMemoryStore()
memory = MemorySaver()


llm = ChatGroq(model='llama-3.2-90b-vision-preview')


trimmer = trim_messages(
    max_tokens=5984,
    strategy="last",
    token_counter=tiktoken_counter,
    include_system=True,
    allow_partial=False,
)

qdrant_retriever = initialize_selfquery_retriever(llm, qdrant_vector_store=qdrant_vector_store)
qdrant_retriever_tool = qdrant_retriever.as_tool(
    name="retrieve_research_paper_texts",
    description="Search and return information from the vector database containing texts of several research papers, and scholarly articles",
)


Started Qdrant client.
Collection 'aireas-cloud' created successfully.
Index created on 'pdf_id' field for efficient filtering.


/tmp/ipykernel_46707/3440341245.py:46: LangChainBetaWarning: This API is in beta and may change in the future.
  qdrant_retriever_tool = qdrant_retriever.as_tool(


In [2]:
decomposer_chain = decomposition_chain(llm=llm)
check_query_chain = requires_decomposition(llm=llm)
rephraser_chain = rephrase_chain(llm=llm)
planner_chain = get_plan_chain(llm=llm)
assign_topic_chain = assign_chat_topic(llm=llm)

In [3]:
assign_topic_chain.invoke(' ill be researching on animation generation using AI, partial attention networks and incorporating diffusion models for frame gen')

'AI-Generated Animation with Diffusion'